Сначала соберём словарь, состоящий из слов формального и неформального датасетов:

In [1]:
class WordTrie:
    '''
        How use 
        wt = WordTrie(FastText())
        wt.build_dict(['word', 'world', 'cat', 'cats'])
        for word, vector in wt.search_by_prefix('cats'): 
            print(word)
    '''
    def __init__(self, word2vec):
        self.root = _Node('*')
        self.get_vector = word2vec.get_vector
        self.model = word2vec

    def add(self, word, is_bigram=False):
        tmp_node = self.root

        for char in word:
            child = tmp_node.children.get(char, None)
            if child is None:
                child = _Node(char)
                tmp_node.children[char] = child
                child.parent = tmp_node
            tmp_node = child
        if not is_bigram and word in self.model.vocab:
            tmp_node.value = self.get_vector(word)
        tmp_node.freq += 1

    def build_dict(self, words):
        for word in words:
            self.add(word)
        return self

    def search_by_prefix(self, prefix):
        tmp_node = self.root

        for char in prefix:
            tmp_node = tmp_node.children.get(char, None)
            if tmp_node is None:
                return
        yield from tmp_node.get_childs()

In [2]:
class _Node:
    def __init__(self, char: str):
        self.char = char
        self.children = {}
        self.value = None
        self.parent = None
        self.freq = 0

    def _get_prefix(self):
        tmp_node = self
        prefix = ''
        while tmp_node.parent:
            prefix = tmp_node.char + prefix
            tmp_node = tmp_node.parent

        return prefix

    def get_childs(self):
        stack = [(self, self._get_prefix())]

        while stack:
            tmp_node, prefix = stack.pop(0)
            ##if tmp_node.value is not None:
            yield prefix, tmp_node.value, tmp_node.freq
            
            for char, child in tmp_node.children.items():
                stack.append((child, prefix+char))

In [3]:
import gensim

w2v_fpath = "all.norm-sz100-w10-cb0-it1-min100.w2v"
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v_model.init_sims(replace=True)
for word, score in w2v_model.most_similar("дерево"):
    print(word, score)

дерево — 0.8678081035614014
деревце 0.867724597454071
деревцо 0.8552745580673218
буковое 0.8502448797225952
дерево, — 0.8466764688491821
дерево… 0.8452415466308594
срубленное 0.8443728685379028
росшее 0.8330211639404297
спиленное 0.826280951499939
развесистое 0.8261986970901489


In [4]:
wt = WordTrie(w2v_model)

In [5]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

stops = stopwords.words("russian")

nlp = spacy.load('ru2')
nlp.add_pipe(nlp.create_pipe('sentencizer'), first=True)

def adding_new_words(dt):
    
    for sentence in dt['defs']:
        text = ''.join(x for x in sentence if x not in punctuation)
        words = word_tokenize(text, language="russian")
        words = [word for word in words if word not in stops]
        text = ' '.join(words)
        doc = nlp(text)
        words = [word.lemma_ for word in nlp(text)]
    
        for word in words:
            wt.add(word)
        
        for i in range(len(words) - 1):
            curr_bigram = words[i] + " " + words[i + 1]
            wt.add(curr_bigram, is_bigram=True)

In [6]:
import pandas as pd

file1 = 'test_set_002.csv'

data = pd.read_csv(file1, delimiter=',')

print(data.shape)
data.head(5)

(26861, 3)


,defs,word,len
0,колпак на лампе,абажур,15
1,настоятель католического монастыря,аббат,34
2,католический священник,аббат,22
3,католический монастырь,аббатство,22
4,слово из первых букв,аббревиатура,20


In [7]:
adding_new_words(data)

In [8]:
file2 = 'test_set_human_003.csv'

human_data = pd.read_csv(file2, comment='#')

print(human_data.shape)
human_data.head(5)

(962, 3)


,defs,word,len
0,легкое агрегатное состояние,газ,58
1,оскорбление самого значимого,святотатство,53
2,разновидность чего-либо,род,42
3,возвышенность наоборот,низина,20
4,скрученная еда,рулет,63


In [9]:
adding_new_words(human_data)

In [10]:
for word, vector, freq in wt.search_by_prefix('кот'): 
    print(word, freq)

кот 2
кото 0
котл 0
котр 0
котё 0
коти 0
кот  0
коте 0
котор 29
котле 0
котло 0
котро 0
котёл 10
котир 0
кот о 0
котел 0
которо 0
которы 0
котор  0
котора 0
котору 0
котлет 0
котлов 0
котрог 0
котёл  0
котиро 0
кот ос 0
котель 0
которое 96
которой 257
котором 182
которог 0
которых 68
которые 55
который 96
которым 50
котор к 0
котор м 0
котор д 0
котор р 0
котор п 0
котор и 0
котор э 0
котор ч 0
котор ц 0
котор с 0
котор ф 0
котор о 0
котор а 0
котор б 0
котор т 0
которая 41
которую 66
котлета 5
котлови 0
котлова 0
котрого 1
котёл п 0
котёл о 0
котёл т 0
котёл у 0
котёл д 0
котиров 0
кот осо 0
котельн 0
которое  0
которой  0
котором  0
которому 69
которого 211
которых  0
которыхс 0
которые  0
который  0
которым  0
которыми 17
котор ко 0
котор кр 0
котор ме 0
котор мо 0
котор де 0
котор ра 0
котор по 0
котор пр 0
котор па 0
котор из 0
котор иг 0
котор эл 0
котор че 0
котор цв 0
котор со 0
котор са 0
котор ст 0
котор фи 0
котор от 0
котор ос 0
котор ат 0
котор ба 0
котор те 0
которая  0
к

котором изо 0
котором изл 0
котором изг 0
котором изз 0
котором исп 0
котором имм 0
котором мол 0
котором муз 0
котором мел 0
котором мас 0
котором сос 0
котором сов 0
котором сод 0
котором соб 0
котором соп 0
котором сох 0
котором спо 0
котором сто 0
котором ста 0
котором стр 0
котором схе 0
котором ска 0
котором два 1
котором дер 0
котором нар 0
котором нах 0
котором нав 0
котором нап 0
котором ник 0
котором нуж 0
котором неп 0
котором нео 0
котором неэ 0
котором нос 0
котором зап 0
котором зас 0
котором зад 0
котором зав 0
котором зол 0
котором жиз 0
котором жид 0
котором афф 0
котором ато 0
котором арт 0
котором хра 0
котором леж 0
котором лиз 0
котором лид 0
котором лун 0
котором лош 0
котором эмо 0
котором шир 0
котором шар 1
котором фун 0
которому пе 0
которому пр 0
которому по 0
которому св 0
которому ст 0
которому сд 0
которому сл 0
которому ск 0
которому съ 0
которому тр 0
которому те 0
которому та 0
которому въ 0
которому вы 0
которому ре 0
которому ра 0
которому до 0
которо

которого нес 0
которого нос 0
которого пос 0
которого под 0
которого пог 0
которого пол 0
которого пом 0
которого пок 0
которого пох 0
которого при 0
которого пре 0
которого про 0
которого пря 0
которого пер 0
которого пес 0
которого пар 0
которого пик 1
которого име 0
которого исп 0
которого исч 0
которого изо 0
которого идт 0
которого гот 0
которого раз 0
которого рав 0
которого уме 0
которого упо 0
которого уча 0
которого явл 0
которого яко 0
которого ест 0
которого леч 0
которого лет 0
которого леж 0
которого лиц 0
которого лош 0
которого люб 0
которого кот 0
которого ком 0
которого кто 0
которого хар 0
которого шам 0
которого фро 0
которого моч 0
которого мен 0
которого жен 0
которого дел 0
которого дол 0
которого два 1
которого арм 0
которого цик 0
которого защ 0
которого заш 0
которого точ 0
которого бла 0
которых чело 0
которых числ 0
которых чтол 0
которых мучи 0
которых печа 0
которых пере 0
которых прох 0
которых прои 0
которых пред 0
которых попе 0
которых позв 0
которых ук

которые касат 0
которые слива 0
которые случа 0
которые являт 0
которые выбит 0
которые возло 0
которые вешае 0
которые владе 0
которые испол 0
которые извес 0
которые иметь 1
которые центр 0
которые указы 0
которые услов 0
которые однов 0
которые обычн 0
которые опред 0
которые гаран 0
который переп 0
который произ 0
который прода 0
который прини 0
который привы 0
который предн 0
который предс 0
который преоб 0
который прави 0
который плати 0
который посту 0
который посте 0
который поджи 0
который получ 0
который позво 0
который стоят 0
который следи 0
который служи 0
который счита 0
который специ 0
который спрое 0
который сорти 0
который созда 0
который суеве 0
который намаз 0
который навеш 0
который наход 0
который нанос 0
который нанес 0
который накла 0
который натян 0
который напис 0
который необх 0
который невоз 0
который разме 0
который работ 0
который религ 0
который решаю 0
который выдел 0
который выраж 0
который выраб 0
который вклад 0
который возгл 0
который воспе 0
который 

которым готовк 0
которым оклика 0
которым холодн 0
которым чтолиб 0
которым какаян 0
которым регуля 0
которыми обесп 0
которыми обычн 0
которыми осыпа 0
которыми оснас 0
которыми высту 0
которыми встре 0
которыми произ 0
которыми покры 0
которыми польз 0
которыми чтоли 0
которыми соеди 0
которыми связы 0
которыми ктоли 0
котор концентр 0
котор крестьян 0
котор действов 0
котор распрост 0
котор подвесит 0
котор приносит 0
котор изучатьс 0
котор цветонож 0
котор фигурный 1
котор отпечаты 0
котор балерина 1
которая выража 0
которая вычисл 0
которая возник 0
которая вместе 1
которая улучши 0
которая упират 0
которая участв 0
которая ударят 0
которая служит 0
которая соотве 0
которая соверш 0
которая содерж 0
которая занима 0
которая заменя 0
которая должны 0
которая делать 1
которая держат 0
которая окаймл 0
которая обеспе 0
которая подорв 0
которая поддер 0
которая понима 0
которая постоя 0
которая повтор 0
которая прелом 0
которая являть 0
которая испыты 0
которая крепит 0
которая развеш

котором обнаружи 0
котором обеспечи 0
котором определи 0
котором отдельны 0
котором регистри 0
котором работать 1
котором различат 0
котором развитие 1
котором развиват 0
котором разъясня 0
котором располаг 0
котором располож 0
котором рассматр 0
котором установи 0
котором устанавл 0
котором участник 1
котором укреплят 0
котором укрепить 1
котором выращива 0
котором выражать 0
котором выполнят 0
котором воздейст 0
котором возможны 0
котором вращение 1
котором вещество 1
котором чествова 0
котором числилос 0
котором теряться 1
котором танцеват 0
котором крепитьс 0
котором качество 1
котором качаться 1
котором изучатьс 0
котором изображе 0
котором изложить 1
котором изготовл 0
котором использо 0
котором исполнят 0
котором иммунный 1
котором молекула 1
котором музыкаль 0
котором маскиров 0
котором составит 0
котором сосредот 0
котором совпадат 0
котором совершат 0
котором содержат 0
котором собирать 0
котором соприкас 0
котором сохранят 0
котором способны 0
котором ставитьс 0
котором стре

которое охватывать 1
которое сообщаются 1
которое сопоставит 0
которое содержимый 1
которое соединятьс 0
которое складывать 0
которое характериз 0
которое необходимы 0
которое невозможны 0
которое толковатьс 0
которое комментиро 0
которое устанавлив 0
которое заведовать 1
которое записывать 0
которое заниматься 1
которое единственн 0
которое действоват 0
которое функционир 0
которое использова 0
которое измельчать 1
которой выстраиват 0
которой вырабатыва 0
которой выигрывать 2
которой воспроизво 0
которой гравировал 0
которой находиться 4
которой находишься 1
которой начинаться 1
которой необходимы 0
которой нижестоящи 0
которой заканчиват 0
которой закреплять 0
которой заниматься 1
которой задаваться 1
которой загибаться 1
которой рассматрив 0
которой расположит 0
которой расположен 0
которой располагат 0
которой размещатьс 0
которой разместить 1
которой разделивши 0
которой равноудале 0
которой регулировщ 0
которой измеряться 2
которой изготовляю 0
которой изображать 0
которой испол

которого осуществляться 1
которого ограничиваться 1
которого приготовляться 1
которого прямоугольники 1
которого переправляются 1
которого паразитировать 1
которого использоваться 3
которого разворачиватьс 0
которого характеризоват 0
которыхсубъектопределяе 0
которые распространятьс 0
которые централизованно 1
который специализируетс 0
которым приспосабливать 0
которыми обеспечиваться 2
котор концентрироваться 1
которая соответствовать 1
которая распространятьс 0
которое характеризоватьс 0
которой воспроизводиться 1
котором регистрироваться 1
котором лизингополучател 0
которому рекомендоваться 1
которому изготавливаются 1
которого разворачиваться 1
которого характеризовать 0
которыхсубъектопределяет 0
которые распространяться 1
который специализируется 1
которым приспосабливатьс 0
которая распространяться 2
которое характеризоваться 1
котором лизингополучатель 1
которого характеризоватьс 0
которыхсубъектопределяетс 0
которым приспосабливаться 1
которого характеризоваться 2
которыхсубъе

Напишем языковую модель на основе построенного словаря:

In [11]:
class LanguageModel: #предполагается, что слова лемматизированы перед тем, как поступают на вход языковой модели
    
    def __init__(self, vocab, n_gram=2, default_proba=0.01):
        self.trie = vocab
        self.def_proba = default_proba
        self.n = n_gram
        
    def update_vocab(self, sentence):
        text = ''.join(x for x in sentence if x not in punctuation)
        words = word_tokenize(text, language="russian")
        words = [word for word in words if word not in stops]
        text = ' '.join(words)
        doc = nlp(text)
        words = [word.lemma_ for word in nlp(text)]
    
        for word in words:
            self.trie.add(word)
        
        for i in range(len(words) - 1):
            curr_bigram = words[i] + " " + words[i + 1]
            self.trie.add(curr_bigram, is_bigram=True)
        
    def calculate_freq(self, word):
        array = self.trie.search_by_prefix(word)
        word_freq = 0
        words = []
        freqs = []
        for w, vec, freq in array:
            words.append(w)
            freqs.append(freq)
        
        for i in range(len(words)):
            if words[i] == word:
                word_freq = freqs[i]
        
        return word_freq
    
    def calculate_bigram_proba(self, bigram):
        bigram_freq = self.calculate_freq(bigram)
        
        if not bigram_freq:
            return self.def_proba
        
        first_word = bigram.split()[0]
        word_freq = self.calculate_freq(first_word)
                
        if not word_freq:
            return default_proba # на самом деле, этот случай возможен, только если и биграмма имеет нулевую частотность
        
        return bigram_freq / word_freq
    
    def calculate_proba(self, sentence):
        proba = 1
        
        text = ''.join(x for x in sentence if x not in punctuation)
        words = word_tokenize(text, language="russian")
        words=[word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stops]
        text = ' '.join(words)
        doc = nlp(text)
        words = [word.lemma_ for word in nlp(text)]
        
        if len(words) < self.n:
            proba = 0
        
        for i in range(len(words) - 1):
            curr_bigram = words[i] + " " + words[i + 1]
            proba *= self.calculate_bigram_proba(curr_bigram)
        
        return proba
            

Обучим модель на построенном словаре, потестируем на примерах:

In [12]:
l_model = LanguageModel(wt)
print(l_model.calculate_proba("Кислый цитрусовый фрукт"))

0.0001


In [13]:
print(l_model.calculate_proba("цитрусовый фрукт"))

0.01


In [14]:
print(l_model.calculate_proba("дед мороз"))

0.01


In [15]:
print(l_model.calculate_freq("дед"))

6


In [16]:
print(l_model.calculate_freq("дед мороз"))

0


In [17]:
print(l_model.calculate_freq("перемещение верхом"))

0


In [18]:
for word, vec, freq in wt.search_by_prefix("перемещение"):
    print(word, freq)

перемещение 78
перемещение  0
перемещение п 0
перемещение к 0
перемещение а 0
перемещение о 0
перемещение е 0
перемещение т 0
перемещение ч 0
перемещение с 0
перемещение в 0
перемещение ж 0
перемещение ц 0
перемещение и 0
перемещение м 0
перемещение н 0
перемещение д 0
перемещение б 0
перемещение г 0
перемещение у 0
перемещение ш 0
перемещение пе 0
перемещение по 0
перемещение пр 0
перемещение ко 0
перемещение ку 0
перемещение ка 0
перемещение ат 0
перемещение от 0
перемещение оч 0
перемещение об 0
перемещение ос 0
перемещение ед 0
перемещение тр 0
перемещение те 0
перемещение та 0
перемещение че 0
перемещение ча 0
перемещение сп 0
перемещение су 0
перемещение св 0
перемещение см 0
перемещение ве 0
перемещение вп 0
перемещение во 0
перемещение жи 0
перемещение це 0
перемещение цв 0
перемещение из 0
перемещение ма 0
перемещение на 0
перемещение до 0
перемещение др 0
перемещение бе 0
перемещение бо 0
перемещение гр 0
перемещение га 0
перемещение ус 0
перемещение ша 0
перемещение пер 0
пе

In [19]:
print(l_model.calculate_freq("перемещение верх"))

2


In [20]:
print(l_model.calculate_proba("перемещение верх"))

0.02564102564102564


In [21]:
print(l_model.calculate_freq("перемещение"))

78


Подсоединим к spell-checker-у языковую модель:

In [22]:
class Spell_checker:
    
    def __init__(self, lang_model, topn=5, max_number_of_errors=10):
        self.trie = lang_model.trie
        self.closest_str = []
        self.lang_model = lang_model
        self.topn = topn #сколько значений возвращает
        self.max_num_err = max_number_of_errors #максимальное количество опечаток, которое 
                                                #может допустить пользователь в одном слове
            
    def update_vocab(self, sentence):
        self.lang_model.update_vocab(sentence)

    def update_values_before_new_word(self):
        self.closest_str = [("", self.max_num_err)] * self.topn #в нём хранятся пары, состоящие из слов и соотв расстояний

    def dfs(self, tmp_node, word, flag): #находит все слова в дереве на расстоянии <= максимального возможного кол-ва опечаток
        # flag indicates whether to use an improved version of the Levenshtein's algorithm
        if tmp_node.value is not None:
            vocab_word = tmp_node._get_prefix()
            number_of_cols = len(vocab_word) + 1
            number_of_rows = len(word) + 1 if not flag else 2
            d_levenshtein = [[0] * number_of_cols for i in range(number_of_rows)]
            for i in range(number_of_rows):
                d_levenshtein[i][0] = i
            for j in range(number_of_cols):
                d_levenshtein[0][j] = j
            for i in range(1, len(word) + 1):
                for j in range(1, number_of_cols):
                    delta = 1
                    if word[i - 1] == vocab_word[j - 1]:
                        delta = 0
                    curr_row = i if not flag else 1
                    d_levenshtein[curr_row][j] = min(d_levenshtein[curr_row][j - 1] + 1, d_levenshtein[curr_row - 1][j] + 1,
                                                     d_levenshtein[curr_row - 1][j - 1] + delta)
                if flag:
                    for j in range(number_of_cols):
                        d_levenshtein[0][j] = d_levenshtein[1][j]
                        d_levenshtein[1][0] += 1
            if d_levenshtein[number_of_rows - 1][len(vocab_word)] <= self.max_num_err:
                curr_dist = d_levenshtein[number_of_rows - 1][len(vocab_word)]
                found_index = -1
                for k in range(self.topn):
                    if curr_dist <= self.closest_str[k][1]:
                        found_index = k
                        break
                if found_index != -1:
                    curr_value = self.closest_str[found_index]
                    for k in range(found_index + 1, self.topn):
                        tmp = self.closest_str[k]
                        self.closest_str[k] = curr_value
                        curr_value = tmp
                        
                    self.closest_str[found_index] = (vocab_word, curr_dist)

        if len(tmp_node.children):
            for child in tmp_node.children.values():
                self.dfs(child, word, flag)

    def search_closest_words(self, word, flag):#ищет ближайшее слово(!) по метрике Левенштейна
        # flag indicates whether to use an improved version of the Levenshtein's algorithm
        self.update_values_before_new_word()
        
        self.dfs(self.trie.root, word, flag)
        
        return sorted(self.closest_str, key=lambda pair: pair[1])
    
    def search_closest_sentence(self, sentence, flag): #в зависимости от длины определения, рассматривается разное количество случаев
        text = ''.join(x for x in sentence if x not in punctuation)
        words = word_tokenize(text, language="russian")
        words=[word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stops]
        text = ' '.join(words)
        doc = nlp(text)
        words = [word.lemma_ for word in nlp(text)]
        
        num_top = 2
        if len(words) <= 5:
            num_top = 5
        elif len(words) <= 10:
            num_top = 3
        
        poss_words = []
        for word in words:
            self.update_values_before_new_word()
            poss_words.append(self.search_closest_words(word, flag)[:num_top])
        
        poss_sent_with_proba = []
        
        def generator(curr_len=0, coord=[0] * len(words)):
            if curr_len == len(words):
                possible_sent = ""
                for i in range(len(words)):
                    x = coord[i]
                    possible_sent += poss_words[i][x][0] + " "
                proba = self.lang_model.calculate_proba(possible_sent)
                poss_sent_with_proba.append((possible_sent, proba))
                return
            else:
                for i in range(num_top):
                    coord[curr_len] = i
                    generator(curr_len + 1, coord)
        
        generator()
        
        sorted(poss_sent_with_proba, key=lambda pair: pair[1], reverse=True)
        closest_sentence = poss_sent_with_proba[0][0]

        return closest_sentence


In [23]:
sp_checker = Spell_checker(l_model)

In [24]:
print(sp_checker.search_closest_sentence("кислый цитрусовый фрукт", False))
print(sp_checker.search_closest_sentence("кислый цитрусовый фрукт", True))

кислый цифровой фрукт 
кислый цифровой фрукт 


In [25]:
tests = [
    "Митрополит возложил на голову Его Императорского Величества ворону",
    "Товарищ Сталин принял польского осла",
    "вызывается в качестве сидетеля",
    "Ячейка обществп",
    "Солнце встанет на двп часа позже",
    "Уставшие гбаза от войнв",
    "Зорко одно шишь сердце",
    "Тестируема исправлерие опечатк",
    "Опечатка опнчатке рознь",
    "Человек чловеку сад",
    "Южный полбс в снегах",
    "Намеренное придумываеие опечатое вредит вашему здоровью:)",
    "По рзелульаттам илссеовадний одонго анлигйсокго унвиертисета, не иеемт занчнеия, в кокам пряокде рсапожолены бкувы в солве",
]

defs_with_err = [
    "это тип двиэения и измееения в прирьде и обществуе, связанныы с переходоп от одного квчества, сосрояния к друшому, от старлго к новому",
    "угол между вртикалью и пооскостью врщения колесс",
    "Ветхое, почтт разваливаюдееся строените",
    "соединени из трёх молекул кислороэа",
    "чистосердечеое признание",
    "чем закрыцвают винр",
    "предмет, на кроторый наручнна проволок",
    "Отпечатлк на песк",
]

In [26]:
for test in tests:
    print(sp_checker.search_closest_sentence(test, True))

митрополит возложить голов императорский величество ворон 
товарищ сталин принять польский посол 
вызывать качестве свидетель 
ячейка общество 
солнце устать два час позже 
устать газа войне 
ярко одно шить сердце 
теорема исправление печать 
перчатка оплате жизнь 
человек человеке сад 
южный полюс снега 
намеренный придумывать оператор вредить ваш здоровье 
результат рассеивание долго английский университет цвет англия рок ряде расположены скулы голове 


In [27]:
for test in defs_with_err:
    print(sp_checker.search_closest_sentence(test, True))

это тип движениях изменении природа общество связанный переходной одного качество состоянии другому строго новый 
угол вертикаль плоскость армения колесо 
ветхий почта разваливаться строение 
соединении три молекула кислород 
чистосердечие признание 
закрывать вино 
предмет который наручник проволока 
отпечаток пуск 


Результаты почти такие же, если всегда рассматривать num_top = 2

In [28]:
class Spell_checker1:
    
    def __init__(self, lang_model, topn=5, max_number_of_errors=10):
        self.trie = lang_model.trie
        self.closest_str = []
        self.lang_model = lang_model
        self.topn = topn #сколько значений возвращает
        self.max_num_err = max_number_of_errors #максимальное количество опечаток, которое 
                                                #может допустить пользователь в одном слове
    
    def update_vocab(self, sentence):
        self.lang_model.update_vocab(sentence)

    def update_values_before_new_word(self):
        self.closest_str = [("", self.max_num_err)] * self.topn #в нём хранятся пары, состоящие из слов и соотв расстояний

    def dfs(self, tmp_node, word, flag): #находит все слова в дереве на расстоянии <= максимального возможного кол-ва опечаток
        # flag indicates whether to use an improved version of the Levenshtein's algorithm
        if tmp_node.value is not None:
            vocab_word = tmp_node._get_prefix()
            number_of_cols = len(vocab_word) + 1
            number_of_rows = len(word) + 1 if not flag else 2
            d_levenshtein = [[0] * number_of_cols for i in range(number_of_rows)]
            for i in range(number_of_rows):
                d_levenshtein[i][0] = i
            for j in range(number_of_cols):
                d_levenshtein[0][j] = j
            for i in range(1, len(word) + 1):
                for j in range(1, number_of_cols):
                    delta = 1
                    if word[i - 1] == vocab_word[j - 1]:
                        delta = 0
                    curr_row = i if not flag else 1
                    d_levenshtein[curr_row][j] = min(d_levenshtein[curr_row][j - 1] + 1, d_levenshtein[curr_row - 1][j] + 1,
                                                     d_levenshtein[curr_row - 1][j - 1] + delta)
                if flag:
                    for j in range(number_of_cols):
                        d_levenshtein[0][j] = d_levenshtein[1][j]
                        d_levenshtein[1][0] += 1
            if d_levenshtein[number_of_rows - 1][len(vocab_word)] <= self.max_num_err:
                curr_dist = d_levenshtein[number_of_rows - 1][len(vocab_word)]
                found_index = -1
                for k in range(self.topn):
                    if curr_dist <= self.closest_str[k][1]:
                        found_index = k
                        break
                if found_index != -1:
                    curr_value = self.closest_str[found_index]
                    for k in range(found_index + 1, self.topn):
                        tmp = self.closest_str[k]
                        self.closest_str[k] = curr_value
                        curr_value = tmp
                        
                    self.closest_str[found_index] = (vocab_word, curr_dist)

        if len(tmp_node.children):
            for child in tmp_node.children.values():
                self.dfs(child, word, flag)

    def search_closest_words(self, word, flag):#ищет ближайшее слово(!) по метрике Левенштейна
        # flag indicates whether to use an improved version of the Levenshtein's algorithm
        self.update_values_before_new_word()
        
        self.dfs(self.trie.root, word, flag)
        
        return sorted(self.closest_str, key=lambda pair: pair[1])
    
    def search_closest_sentence(self, sentence, flag, num_top=2): #!!! Для каждого слова рассматриваю по 2 первых варианта (из расчёта, что 2^20 ~10^6)
        text = ''.join(x for x in sentence if x not in punctuation)
        words = word_tokenize(text, language="russian")
        words=[word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stops]
        text = ' '.join(words)
        doc = nlp(text)
        words = [word.lemma_ for word in nlp(text)]
        
        poss_words = []
        for word in words:
            self.update_values_before_new_word()
            poss_words.append(self.search_closest_words(word, flag)[:num_top])
        
        poss_sent_with_proba = []
        
        def generator(curr_len=0, coord=[0] * len(words)):
            if curr_len == len(words):
                possible_sent = ""
                for i in range(len(words)):
                    x = coord[i]
                    possible_sent += poss_words[i][x][0] + " "
                proba = self.lang_model.calculate_proba(possible_sent)
                poss_sent_with_proba.append((possible_sent, proba))
                return
            else:
                for i in range(num_top):
                    coord[curr_len] = i
                    generator(curr_len + 1, coord)
        
        generator()
        
        sorted(poss_sent_with_proba, key=lambda pair: pair[1], reverse=True)
        closest_sentence = poss_sent_with_proba[0][0]

        return closest_sentence


In [29]:
sp_checker1 = Spell_checker1(l_model)
print(sp_checker1.search_closest_sentence("кислый цитрусовый фрукт", False))
print(sp_checker1.search_closest_sentence("кислый цитрусовый фрукт", True))

кислый цифровой фрукт 
кислый цифровой фрукт 


In [30]:
for test in tests:
    print(sp_checker1.search_closest_sentence(test, True))

митрополит возложить голов императорский величество ворон 
товарищ сталин принять польский посол 
вызывать качестве свидетель 
ячейка общество 
солнце устать два час позже 
устать газа войне 
ярко одно шить сердце 
теорема исправление печать 
перчатка оплате жизнь 
человек человеке сад 
южный полюс снега 
намеренный придумывать оператор вредить ваш здоровье 
результат рассеивание долго английский университет цвет англия рок ряде расположены скулы голове 


In [31]:
for test in defs_with_err:
    print(sp_checker1.search_closest_sentence(test, True))

это тип движениях изменении природа общество связанный переходной одного качество состоянии другому строго новый 
угол вертикаль плоскость армения колесо 
ветхий почта разваливаться строение 
соединении три молекула кислород 
чистосердечие признание 
закрывать вино 
предмет который наручник проволока 
отпечаток пуск 


Сохраним модель:

In [32]:
import pickle

with open('spell_checker.pickle', 'wb') as file:
    pickle.dump(sp_checker, file)
    
with open('spell_checker1.pickle', 'wb') as file:
    pickle.dump(sp_checker1, file)